In [346]:
import pandas as pd
import numpy as np

import warnings

from tqdm import tqdm

warnings.filterwarnings("ignore")

In [313]:
DATA_PATH = "../data/GSE69529_metadata.csv"

data = pd.read_csv(DATA_PATH)
data.head()

,lib_id,sample_name,infection_group,"age, months",age_group,who_score,sex,"height, m","weight, kg","wbc, 10^6/L",...,"lymphocytes, 10^6/L","monocytes, 10^6/L","eosinophils, 10^6/L","basophils, 10^6/L","red blood cell, 10^6/uL","platelets, cells/uL",highest_temp,duration_days,days_past_onset,rotavirus vaccine doses
0,lib6081,CO061,HC,11,<2y,Control,female,0.9,12.7,10330,...,3900,480,280,110,3.98,370890,36.5,NaN,NaN,2
1,lib6105,CO062,HC,48,2y+,Control,female,0.91,13.0,10330,...,3900,480,280,110,3.98,370890,36.6,NaN,NaN,0
2,lib6129,CO063,HC,48,2y+,Control,female,0.9,14.9,6500,...,3480,420,310,50,4.14,379000,36.5,NaN,NaN,0
3,lib6153,CO024,HC,15,<2y,Control,male,0.77,11.8,11310,...,5900,580,180,100,3.46,261000,36.6,NaN,NaN,2
4,lib6177,CO013,HC,16,<2y,Control,male,1.2,15.0,5570,...,3800,30,20,80,4.75,370940,35.7,NaN,NaN,2


## Data Cleaning & Preprocessing

In [314]:
# remove HC in infection_group
data = data[data["infection_group"] != "HC"].reset_index(drop=True)

# remove redundant column = [age_group]
data.drop(columns=["age_group"], inplace=True)

# remove unused columns = [lib_id, sample_name]
data.drop(columns=["lib_id", "sample_name"], inplace=True)

print(data["infection_group"].unique())
print(data.head(3))

['Ecoli' 'Salmonella' 'Shigella' 'Rotavirus']
  infection_group  age, months who_score     sex height, m  weight, kg  \
0           Ecoli           12      mild  female      0.81        10.3   
1           Ecoli           24      mild    male      0.84        12.9   
2           Ecoli           24    severe  female      0.82         5.8   

   wbc, 10^6/L  neutrophils, 10^6/L  lymphocytes, 10^6/L  monocytes, 10^6/L  \
0        21890                17380                 2070               1160   
1        23900                11980                 6790               4900   
2        12600                 8830                 2710                860   

   eosinophils, 10^6/L  basophils, 10^6/L  red blood cell, 10^6/uL  \
0                  260                220                     4.07   
1                  150                 70                     2.99   
2                  100                100                     4.35   

   platelets, cells/uL  highest_temp  duration_days  days_p

In [315]:
data[data["duration_days"].isna()]

,infection_group,"age, months",who_score,sex,"height, m","weight, kg","wbc, 10^6/L","neutrophils, 10^6/L","lymphocytes, 10^6/L","monocytes, 10^6/L","eosinophils, 10^6/L","basophils, 10^6/L","red blood cell, 10^6/uL","platelets, cells/uL",highest_temp,duration_days,days_past_onset,rotavirus vaccine doses
41,Ecoli,3,moderate,female,0.55,4.4,12000,2700,8100,1200,0,0,3.31,376000,39.0,NaN,2.0,1
90,Shigella,15,moderate,male,0.76,9.4,7800,2300,4300,1200,0,0,4.06,393000,39.5,NaN,8.0,0


In [316]:
data[data["highest_temp"].isna()]

,infection_group,"age, months",who_score,sex,"height, m","weight, kg","wbc, 10^6/L","neutrophils, 10^6/L","lymphocytes, 10^6/L","monocytes, 10^6/L","eosinophils, 10^6/L","basophils, 10^6/L","red blood cell, 10^6/uL","platelets, cells/uL",highest_temp,duration_days,days_past_onset,rotavirus vaccine doses
61,Salmonella,48,mild,female,0.78,14.0,3860,2010,1470,210,150,20,4.12,259000,NaN,7.0,1.0,2


In [317]:
# Data Imputation with KNNImputer

from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)
data_imputed = imputer.fit_transform(data[["duration_days", "highest_temp"]])
data_imputed = pd.DataFrame(data_imputed, columns=["duration_days", "highest_temp"])

data["duration_days"] = data_imputed["duration_days"]
data["highest_temp"] = data_imputed["highest_temp"]

data.isna().sum()

infection_group            0
age, months                0
who_score                  0
sex                        0
height, m                  0
weight, kg                 0
wbc, 10^6/L                0
neutrophils, 10^6/L        0
lymphocytes, 10^6/L        0
monocytes, 10^6/L          0
eosinophils, 10^6/L        0
basophils, 10^6/L          0
red blood cell, 10^6/uL    0
platelets, cells/uL        0
highest_temp               0
duration_days              0
days_past_onset            0
rotavirus vaccine doses    0
dtype: int64

In [318]:
# Handling Incosisntent Data

data["height, m"] = data["height, m"].replace(
    {
        "0.76 CM ":"0.76",
        '100': '1.0',
        '90.5': '0.905',
     }
)
data["height, m"] = data["height, m"].astype(float)

height_unique = list(data["height, m"].unique())
height_unique.sort()
print(height_unique)

[0.51, 0.53, 0.54, 0.55, 0.57, 0.58, 0.59, 0.6, 0.61, 0.62, 0.63, 0.64, 0.65, 0.67, 0.68, 0.7, 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.8, 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 0.905, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0, 1.02, 1.04, 1.05, 1.08, 1.09, 1.1, 1.13, 1.14, 1.15, 1.16, 1.17, 1.19, 1.2, 1.24, 1.25, 1.26, 1.4, 1.45, 1.6]


In [319]:
data

,infection_group,"age, months",who_score,sex,"height, m","weight, kg","wbc, 10^6/L","neutrophils, 10^6/L","lymphocytes, 10^6/L","monocytes, 10^6/L","eosinophils, 10^6/L","basophils, 10^6/L","red blood cell, 10^6/uL","platelets, cells/uL",highest_temp,duration_days,days_past_onset,rotavirus vaccine doses
0,Ecoli,12,mild,female,0.81,10.3,21890,17380,2070,1160,260,220,4.07,274000,36.0,4.0,1.0,2
1,Ecoli,24,mild,male,0.84,12.9,23900,11980,6790,4900,150,70,2.99,340000,39.0,5.0,5.0,1
2,Ecoli,24,severe,female,0.82,5.8,12600,8830,2710,860,100,100,4.35,461000,39.0,7.0,3.0,0
3,Ecoli,6,mild,male,0.62,6.0,15030,6840,6600,1230,180,180,3.77,361000,37.0,3.0,1.0,0
4,Ecoli,11,mild,male,0.72,8.0,17350,8880,6250,1560,330,330,4.49,518000,36.5,1.0,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,Rotavirus,18,mild,female,0.80,12.4,12800,10200,1500,1000,0,0,4.24,483000,38.0,4.0,2.0,3
160,Rotavirus,60,mild,male,0.99,17.5,9900,8200,800,900,0,0,4.70,346000,37.0,6.0,2.0,0
161,Rotavirus,22,mild,male,0.85,14.3,5900,2700,2300,800,0,0,5.25,383000,37.5,8.0,4.0,0
162,Rotavirus,4,severe,male,0.55,4.5,20600,10000,8200,2300,0,0,3.33,556000,37.9,13.0,8.0,2


In [320]:
num_kolom = data.select_dtypes(include=['float64', 'int64']).columns

In [321]:
# 0 untuk virus dan 1 untuk bakteri
dict_infection_group = {
    "Ecoli": 1,
    "Salmonella": 1,
    "Shigella": 1,
    "Rotavirus": 0,
}

data["infection_group"] = data["infection_group"].map(dict_infection_group)

data.head()

,infection_group,"age, months",who_score,sex,"height, m","weight, kg","wbc, 10^6/L","neutrophils, 10^6/L","lymphocytes, 10^6/L","monocytes, 10^6/L","eosinophils, 10^6/L","basophils, 10^6/L","red blood cell, 10^6/uL","platelets, cells/uL",highest_temp,duration_days,days_past_onset,rotavirus vaccine doses
0,1,12,mild,female,0.81,10.3,21890,17380,2070,1160,260,220,4.07,274000,36.0,4.0,1.0,2
1,1,24,mild,male,0.84,12.9,23900,11980,6790,4900,150,70,2.99,340000,39.0,5.0,5.0,1
2,1,24,severe,female,0.82,5.8,12600,8830,2710,860,100,100,4.35,461000,39.0,7.0,3.0,0
3,1,6,mild,male,0.62,6.0,15030,6840,6600,1230,180,180,3.77,361000,37.0,3.0,1.0,0
4,1,11,mild,male,0.72,8.0,17350,8880,6250,1560,330,330,4.49,518000,36.5,1.0,2.0,0


## Feature Engineering

In [322]:
dict_who_score = {
    "mild": 1,
    "moderate": 2,
    "severe": 3,
}

data["who_score"] = data["who_score"].map(dict_who_score)

dict_sex = {
    "female": 0,
    "male": 1
}

data["sex"] = data["sex"].map(dict_sex)

data.head()

,infection_group,"age, months",who_score,sex,"height, m","weight, kg","wbc, 10^6/L","neutrophils, 10^6/L","lymphocytes, 10^6/L","monocytes, 10^6/L","eosinophils, 10^6/L","basophils, 10^6/L","red blood cell, 10^6/uL","platelets, cells/uL",highest_temp,duration_days,days_past_onset,rotavirus vaccine doses
0,1,12,1,0,0.81,10.3,21890,17380,2070,1160,260,220,4.07,274000,36.0,4.0,1.0,2
1,1,24,1,1,0.84,12.9,23900,11980,6790,4900,150,70,2.99,340000,39.0,5.0,5.0,1
2,1,24,3,0,0.82,5.8,12600,8830,2710,860,100,100,4.35,461000,39.0,7.0,3.0,0
3,1,6,1,1,0.62,6.0,15030,6840,6600,1230,180,180,3.77,361000,37.0,3.0,1.0,0
4,1,11,1,1,0.72,8.0,17350,8880,6250,1560,330,330,4.49,518000,36.5,1.0,2.0,0


In [323]:
# ratio calculation

data["NLR"] = data["neutrophils, 10^6/L"] / data["lymphocytes, 10^6/L"]
data["PLR"] = data["platelets, cells/uL"] / data["lymphocytes, 10^6/L"]
data["MLR"] = data["monocytes, 10^6/L"] / data["lymphocytes, 10^6/L"]
data["NMR"] = data["neutrophils, 10^6/L"] / data["monocytes, 10^6/L"]
data["NPR"] = data["neutrophils, 10^6/L"] / data["platelets, cells/uL"]
data["LMR"] = data["lymphocytes, 10^6/L"] / data["monocytes, 10^6/L"]
data["red2white"] = data["red blood cell, 10^6/uL"] / data["wbc, 10^6/L"]

num_kolom = num_kolom.tolist()
num_kolom += ["NLR", "PLR", "MLR", "NMR", "NPR", "LMR", "red2white"]

data.head()

,infection_group,"age, months",who_score,sex,"height, m","weight, kg","wbc, 10^6/L","neutrophils, 10^6/L","lymphocytes, 10^6/L","monocytes, 10^6/L",...,days_past_onset,rotavirus vaccine doses,NLR,PLR,MLR,NMR,NPR,LMR,red2white,age_group
0,1,12,1,0,0.81,10.3,21890,17380,2070,1160,...,1.0,2,8.396135,132.367150,0.560386,14.982759,0.063431,1.784483,0.000186,1
1,1,24,1,1,0.84,12.9,23900,11980,6790,4900,...,5.0,1,1.764359,50.073638,0.721649,2.444898,0.035235,1.385714,0.000125,3
2,1,24,3,0,0.82,5.8,12600,8830,2710,860,...,3.0,0,3.258303,170.110701,0.317343,10.267442,0.019154,3.151163,0.000345,3
3,1,6,1,1,0.62,6.0,15030,6840,6600,1230,...,1.0,0,1.036364,54.696970,0.186364,5.560976,0.018947,5.365854,0.000251,1
4,1,11,1,1,0.72,8.0,17350,8880,6250,1560,...,2.0,0,1.420800,82.880000,0.249600,5.692308,0.017143,4.006410,0.000259,1


In [324]:
data.isna().sum()

infection_group            0
age, months                0
who_score                  0
sex                        0
height, m                  0
weight, kg                 0
wbc, 10^6/L                0
neutrophils, 10^6/L        0
lymphocytes, 10^6/L        0
monocytes, 10^6/L          0
eosinophils, 10^6/L        0
basophils, 10^6/L          0
red blood cell, 10^6/uL    0
platelets, cells/uL        0
highest_temp               0
duration_days              0
days_past_onset            0
rotavirus vaccine doses    0
NLR                        0
PLR                        0
MLR                        0
NMR                        0
NPR                        0
LMR                        0
red2white                  0
age_group                  0
dtype: int64

## Desain Evaluasi

In [348]:
# stratified k-fold
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
from sklearn.preprocessing import StandardScaler

# Adjustable baseline model independent variables
# - oversampling or not
# - which oversampling method
# - log transformation or not
# - scaling or not
# - which scaling method
# all combinations scaling too

possible_logs = [False, True]
possible_oversampling = [False, True]
possible_oversampling_method = [SMOTE, ADASYN, RandomOverSampler]
possible_scaling = [False, True]

# if oversampling is False, then oversampling_method is None
# if scaling is False, then scaling_method is None
scenarios = []
for log in possible_logs:
    for oversampling in possible_oversampling:
        for oversampling_method in possible_oversampling_method:
            if not oversampling:
                oversampling_method = None
            scenarios.append({
                "log": log,
                "oversampling": oversampling,
                "oversampling_method": oversampling_method,
            })

# Control variables
N_SPLIT = 3
RANDOM_STATE = 42
SHUFFLE = True
CLASSIFIER = RandomForestClassifier(random_state=RANDOM_STATE)

kfold = StratifiedKFold(n_splits=N_SPLIT, shuffle=SHUFFLE, random_state=RANDOM_STATE)
X, y= data.drop(columns=["infection_group"]), data["infection_group"]

# train
results_df = pd.DataFrame()
for idx, scenario in enumerate(scenarios):
    print(f"Scenario {idx}: {scenario}")
    accs = []
    precs = []
    recs = []
    f1s = []
    aucs = []

    for idx_k, (train_index, test_index) in enumerate(kfold.split(X, y)):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        scaler = StandardScaler()
        X_train[num_kolom] = scaler.fit_transform(X_train[num_kolom])
        X_test[num_kolom] = scaler.transform(X_test[num_kolom])
        
        if scenario["log"]:
            X_train = X_train.apply(lambda x: np.log(x) if x.name in num_kolom and x.min() > 0 else x)
            X_test = X_test.apply(lambda x: np.log(x) if x.name in num_kolom and x.min() > 0 else x)

        if scenario["oversampling"]:
            oversampler = scenario["oversampling_method"](random_state=RANDOM_STATE)
            X_train, y_train = oversampler.fit_resample(X_train, y_train)

        CLASSIFIER.fit(X_train, y_train)
        y_pred = CLASSIFIER.predict(X_test)

        k_acc = accuracy_score(y_test, y_pred)
        k_prec = precision_score(y_test, y_pred)
        k_rec = recall_score(y_test, y_pred)
        k_f1 = f1_score(y_test, y_pred)
        k_fpr, k_tpr, k_thresholds = roc_curve(y_test, y_pred)
        k_auc = auc(k_fpr, k_tpr)
        
        accs.append(k_acc)
        precs.append(k_prec)
        recs.append(k_rec)
        f1s.append(k_f1)
        aucs.append(k_auc)
        
        print(f"Fold {idx_k+1}")
        print(f"Accuracy: {k_acc}")
        print(f"Precision: {k_prec}")
        print(f"Recall: {k_rec}")
        print(f"F1: {k_f1}")
        print(f"AUC: {k_auc}")
        print("="*10)
        print()
    
    mean_accs = np.mean(accs)
    mean_precs = np.mean(precs)
    mean_recs = np.mean(recs)
    mean_f1s = np.mean(f1s)
    mean_aucs = np.mean(aucs)
    
    print(f"Mean Accuracy: {mean_accs}")
    print(f"Mean Precision: {mean_precs}")
    print(f"Mean Recall: {mean_recs}")
    print(f"Mean F1: {mean_f1s}")
    print(f"Mean AUC: {mean_aucs}")
    print("="*10)
    print()
    
    results_df = pd.concat([results_df, pd.DataFrame({
        "scenario_id": [idx],
        "scenario": [scenario],
        "accuracy": [mean_accs],
        "precision": [mean_precs],
        "recall": [mean_recs],
        "f1": [mean_f1s],
        "auc": [mean_aucs],
    })])

Scenario 0: {'log': False, 'oversampling': False, 'oversampling_method': None}
Fold 1
Accuracy: 0.7818181818181819
Precision: 0.8095238095238095
Recall: 0.8947368421052632
F1: 0.85
AUC: 0.7120743034055728

Fold 2
Accuracy: 0.7818181818181819
Precision: 0.7777777777777778
Recall: 0.9459459459459459
F1: 0.8536585365853658
AUC: 0.6951951951951952

Fold 3
Accuracy: 0.7777777777777778
Precision: 0.8378378378378378
Recall: 0.8378378378378378
F1: 0.8378378378378378
AUC: 0.7424483306836247

Mean Accuracy: 0.7804713804713805
Mean Precision: 0.8083798083798084
Mean Recall: 0.8928402086296824
Mean F1: 0.8471654581410678
Mean AUC: 0.7165726097614643

Scenario 1: {'log': False, 'oversampling': False, 'oversampling_method': None}
Fold 1
Accuracy: 0.7818181818181819
Precision: 0.8095238095238095
Recall: 0.8947368421052632
F1: 0.85
AUC: 0.7120743034055728

Fold 2
Accuracy: 0.7818181818181819
Precision: 0.7777777777777778
Recall: 0.9459459459459459
F1: 0.8536585365853658
AUC: 0.6951951951951952

Fold 3

In [350]:
results_df.sort_values(by="auc", ascending=False, inplace=True)
results_df.reset_index(drop=True, inplace=True)

results_df.to_csv("experiments/feature_engineering_results.csv", index=False)

pemenang: {'log': False, 'oversampling': True, 'oversampling_method': <class 'imblearn.over_sampling._random_over_sampler.RandomOverSampler'>}

dengan AUC 79%

In [352]:
results_df.iloc[0]["scenario"]

{'log': False,
 'oversampling': True,
 'oversampling_method': imblearn.over_sampling._random_over_sampler.RandomOverSampler}

In [351]:
# reproduce best scenario training
best_scenario = {
    'log': False,
    'oversampling': True,
    'oversampling_method': RandomOverSampler
}

print(f"Reproducing best scenario: {best_scenario}")

accs = []
precs = []
recs = []
f1s = []
aucs = []

for idx_k, (train_index, test_index) in enumerate(kfold.split(X, y)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    scaler = StandardScaler()
    X_train[num_kolom] = scaler.fit_transform(X_train[num_kolom])
    X_test[num_kolom] = scaler.transform(X_test[num_kolom])

    if best_scenario["log"]:
        X_train = X_train.apply(lambda x: np.log(x) if x.name in num_kolom and x.min() > 0 else x)
        X_test = X_test.apply(lambda x: np.log(x) if x.name in num_kolom and x.min() > 0 else x)

    if best_scenario["oversampling"]:
        oversampler = best_scenario["oversampling_method"](random_state=RANDOM_STATE)
        X_train, y_train = oversampler.fit_resample(X_train, y_train)

    CLASSIFIER.fit(X_train, y_train)
    y_pred = CLASSIFIER.predict(X_test)

    k_acc = accuracy_score(y_test, y_pred)
    k_prec = precision_score(y_test, y_pred)
    k_rec = recall_score(y_test, y_pred)
    k_f1 = f1_score(y_test, y_pred)
    k_fpr, k_tpr, k_thresholds = roc_curve(y_test, y_pred)
    k_auc = auc(k_fpr, k_tpr)

    accs.append(k_acc)
    precs.append(k_prec)
    recs.append(k_rec)
    f1s.append(k_f1)
    aucs.append(k_auc)

    print(f"Fold {idx_k+1}")
    print(f"Accuracy: {k_acc}")
    print(f"Precision: {k_prec}")
    print(f"Recall: {k_rec}")
    print(f"F1: {k_f1}")
    print(f"AUC: {k_auc}")
    print("="*10)
    print()

mean_accs = np.mean(accs)
mean_precs = np.mean(precs)
mean_recs = np.mean(recs)
mean_f1s = np.mean(f1s)
mean_aucs = np.mean(aucs)

print(f"Mean Accuracy: {mean_accs}")
print(f"Mean Precision: {mean_precs}")
print(f"Mean Recall: {mean_recs}")
print(f"Mean F1: {mean_f1s}")
print(f"Mean AUC: {mean_aucs}")
print("="*10)

Reproducing best scenario: {'log': False, 'oversampling': True, 'oversampling_method': <class 'imblearn.over_sampling._random_over_sampler.RandomOverSampler'>}
Fold 1
Accuracy: 0.8363636363636363
Precision: 0.8536585365853658
Recall: 0.9210526315789473
F1: 0.8860759493670886
AUC: 0.7840557275541795

Fold 2
Accuracy: 0.8363636363636363
Precision: 0.8181818181818182
Recall: 0.972972972972973
F1: 0.8888888888888888
AUC: 0.7642642642642643

Fold 3
Accuracy: 0.8333333333333334
Precision: 0.8888888888888888
Recall: 0.8648648648648649
F1: 0.8767123287671232
AUC: 0.814785373608903

Mean Accuracy: 0.8353535353535353
Mean Precision: 0.8535764145520243
Mean Recall: 0.9196301564722619
Mean F1: 0.8838923890077002
Mean AUC: 0.7877017884757823


In [355]:
data.to_csv("../data/preprocessed/feature_engineered_data.csv", index=False)